# Hot Day Prediction Pipeline

This notebook implements a machine learning pipeline for predicting hot days using temperature data from GRIB files.

In [1]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import xarray as xr
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE

In [2]:
def load_and_preprocess_data(grib_file_path, low_temp_threshold=285, high_temp_percentile=0.95, window_size=7):
    """
    Load and preprocess GRIB dataset
    
    Args:
        grib_file_path (str): Path to GRIB file
        low_temp_threshold (float): Threshold for low temperature anomalies
        high_temp_percentile (float): Percentile for defining hot days
        window_size (int): Rolling window size for heatwave detection
    
    Returns:
        tuple: Processed temperature data and percentile
    """
    # Load dataset
    data = xr.open_dataset(grib_file_path, engine="cfgrib")
    
    # Select region and time period
    data_region = data.sel(
        latitude=slice(43, 38),  # From 43°N to 38°N
        longitude=slice(-8, -1)  # From 8°W to 1°W
    )
    data_summer = data_region.sel(time=data_region["time"].dt.month.isin([5, 6, 7, 8, 9]))

    # Extract temperature at noon
    temperature_summer = data_summer["t2m"]
    temp_summer_12 = temperature_summer.sel(time=temperature_summer["time"].dt.hour == 12)
    temp_summer_12 = temp_summer_12.assign_coords(day=temp_summer_12["time"].dt.floor("D"))
    temp_summer_12 = temp_summer_12.swap_dims({"time": "day"}).reset_coords("time", drop=True)

    # Convert to DataFrame
    temp_data = temp_summer_12.to_dataframe().reset_index()
    
    # Filter out consistently low-temperature locations
    mean_temp_by_location = temp_data.groupby(["latitude", "longitude"])["t2m"].mean()
    low_temp_locations = mean_temp_by_location[mean_temp_by_location <= low_temp_threshold].index
    temp_data = temp_data[~temp_data.set_index(["latitude", "longitude"]).index.isin(low_temp_locations)].reset_index(drop=True)

    # Calculate percentiles and hot day labels
    percentile_95 = temp_data["t2m"].quantile(high_temp_percentile)
    
    # Generate hot day labels
    hot_days = temp_data["t2m"] > percentile_95
    hot_days_rolling = hot_days.rolling(window=window_size, center=False).sum()
    labels_next_7_days = (hot_days_rolling >= 3).astype(int)
    
    # Align labels with the dataset
    temp_data["heatwave_label"] = labels_next_7_days.shift(-window_size + 1).fillna(0).astype(int)
    
    return temp_data, percentile_95

In [3]:
def detect_temperature_anomalies(temp_data):
    """
    Detect temperature anomalies using Isolation Forest over the entire dataset.

    Args:
        temp_data (pd.DataFrame): Full temperature dataset.
        contamination (float): Proportion of anomalies expected in the dataset.

    Returns:
        pd.DataFrame: Dataset with anomaly flags.
    """
    # Copy data to avoid modifying the original
    temp_data_copy = temp_data.copy()

    # Prepare data for Isolation Forest (only numerical features)
    features = temp_data_copy[["t2m"]]
    
    # Train Isolation Forest
    anomaly_detector = IsolationForest(
        contamination=0.01, random_state=42
    )
    anomaly_detector.fit(features)

    # Predict anomalies
    temp_data_copy["is_anomaly"] = anomaly_detector.predict(features) == -1  # -1 indicates anomaly

    return temp_data_copy


In [4]:
def prepare_features(data, feature_window_size=30, window_size=7):
    """
    Prepare features for machine learning models
    
    Args:
        data (pd.DataFrame): Processed temperature data
        feature_window_size (int): Size of feature rolling window
        window_size (int): Size of heatwave prediction window
    
    Returns:
        tuple: Features and labels
    """
    X = []
    y = []
    
    # Filter out anomalous data
    normal_data = data[~data["is_anomaly"]]
    
    unique_locations = normal_data[["latitude", "longitude"]].drop_duplicates()
    
    for _, location in unique_locations.iterrows():
        location_data = normal_data[
            (normal_data["latitude"] == location["latitude"]) & 
            (normal_data["longitude"] == location["longitude"])
        ].sort_values("day")
        
        # Percentile for this location from the entire dataset
        location_percentile_95 = data["t2m"].quantile(0.95)
        
        # Create rolling windows of features
        for i in range(len(location_data) - feature_window_size - window_size + 1):
            features = location_data.iloc[i:i+feature_window_size]["t2m"].values
            label = location_data.iloc[i+feature_window_size+window_size-1]["heatwave_label"]
            
            X.append(np.concatenate([
                features, 
                [location["latitude"], location["longitude"], location_percentile_95]
            ]))
            y.append(label)
    
    return np.array(X), np.array(y)

In [5]:
def create_neural_network(input_size):
    """
    Create a neural network model for hot day prediction
    
    Args:
        input_size (int): Number of input features
    
    Returns:
        nn.Module: Neural network model
    """
    class HotDayPredictor(nn.Module):
        def __init__(self, input_size):
            super().__init__()
            self.fc = nn.Sequential(
                nn.Linear(input_size, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        
        def forward(self, x):
            return self.fc(x)
    
    return HotDayPredictor(input_size)

In [6]:
def train_neural_network(model, X_train, y_train, epochs=30, batch_size=32):
    """
    Train the neural network
    
    Args:
        model (nn.Module): Neural network model
        X_train (np.ndarray): Training features
        y_train (np.ndarray): Training labels
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
    
    Returns:
        nn.Module: Trained neural network
    """
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    
    # Create dataset and dataloader
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Training setup
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()
    
    # Train the model
    model.train()
    for epoch in range(epochs):
        for batch_features, batch_labels in dataloader:
            predictions = model(batch_features).squeeze()
            loss = loss_fn(predictions, batch_labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    return model

In [7]:
def train_models(X_train, y_train):
    """
    Train multiple machine learning models
    
    Args:
        X_train (np.ndarray): Training features
        y_train (np.ndarray): Training labels
    
    Returns:
        tuple: Trained models and scaler
    """

    # Balance training data with SMOTE
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    # Standardize features
    scaler = StandardScaler()
    X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)
    
    # Initialize models
    models = {
        'Logistic Regression': LogisticRegression(max_iter=500, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10)
        #'Neural Network': create_neural_network(X_train.shape[1])
    }
    
    # Train models
    for name, model in models.items():
        if name == 'Neural Network':
            # Neural network training
            model = train_neural_network(model, X_train_balanced_scaled, y_train_balanced)
        else:
            # Sklearn models
            model.fit(X_train_balanced_scaled, y_train_balanced)
        
        models[name] = model
    
    return models, scaler

In [8]:
def evaluate_models(models, scaler, X_test, y_test):
    """
    Evaluate trained models
    
    Args:
        models (dict): Trained models
        scaler (StandardScaler): Feature scaler
        X_test (np.ndarray): Test features
        y_test (np.ndarray): Test labels
    
    Returns:
        dict: Model performance metrics
    """
    # Standardize test features
    X_test_scaled = scaler.transform(X_test)
    
    # Evaluation metrics
    results = {}
    
    for name, model in models.items():
        # Predictions
        if name == 'Neural Network':
            # For neural network
            model.eval()
            with torch.no_grad():
                X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
                predictions = model(X_test_tensor).squeeze().numpy()
                predicted_classes = (predictions > 0.5).astype(int)
        else:
            # For sklearn models
            predicted_classes = model.predict(X_test_scaled)
        
        # Calculate metrics
        results[name] = {
            'accuracy': accuracy_score(y_test, predicted_classes),
            'precision': precision_score(y_test, predicted_classes),
            'recall': recall_score(y_test, predicted_classes),
            'f1_score': f1_score(y_test, predicted_classes)
        }
    
    return results

In [9]:
def run_hot_day_prediction_pipeline(grib_file_path, test_size=0.2):
    """
    Run the complete hot day prediction pipeline
    
    Args:
        grib_file_path (str): Path to GRIB file
        test_size (float): Proportion of data to use for testing
    
    Returns:
        tuple: Model results, trained models, and scaler
    """
    # Preprocess data
    temp_data, percentile_95 = load_and_preprocess_data(grib_file_path)
    
    # Detect anomalies
    temp_data_with_anomalies = detect_temperature_anomalies(temp_data)
    
    # Prepare features
    X, y = prepare_features(temp_data_with_anomalies)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Train models
    models, scaler = train_models(X_train, y_train)
    
    # Evaluate models
    results = evaluate_models(models, scaler, X_test, y_test)
    
    return results, models, scaler

In [10]:
# Main execution
# Path to GRIB file
grib_file_path = './spain_may2sept.grib'

# Run pipeline
results, models, scaler = run_hot_day_prediction_pipeline(grib_file_path)

# Print results
for model_name, metrics in results.items():
    print(f'\n{model_name} Performance:')
    for metric, value in metrics.items():
        print(f'{metric.capitalize()}: {value:.4f}')

skipping variable: paramId==235033 shortName='msshf'
Traceback (most recent call last):
  File "c:\Users\Ana\anaconda3\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "c:\Users\Ana\anaconda3\Lib\site-packages\cfgrib\dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1143849600, 1143892800, 1143936000, ..., 1727611200, 1727654400,
       1727697600])) new_value=Variable(dimensions=('time',), data=array([1143828000, 1143871200, 1143914400, ..., 1727589600, 1727632800,
       1727676000]))
skipping variable: paramId==147 shortName='slhf'
Traceback (most recent call last):
  File "c:\Users\Ana\anaconda3\Lib\site-packages\cfgrib\dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "c:\Users\Ana\anaconda3\Lib\site-packages\cfgrib\d


Logistic Regression Performance:
Accuracy: 0.7449
Precision: 0.1586
Recall: 0.8116
F1_score: 0.2653

Random Forest Performance:
Accuracy: 0.8019
Precision: 0.2089
Recall: 0.8932
F1_score: 0.3386
